In [1]:
import os
os.getcwd()
os.chdir("/scratch/jq2uw/derm_vlms")
os.getcwd()

'/sfs/weka/scratch/jq2uw/derm_vlms'

In [2]:
import os, torch
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'
torch.cuda.empty_cache()

from data_utils import *


In [3]:
# --- init data ---
df = process_tabular("./data")
train_df, val_df, test_df = split_df(df)
train_dataset = MIDASDataset(train_df, "./data")
val_dataset = MIDASDataset(val_df, "./data")
test_dataset = MIDASDataset(test_df, "./data")

id_patient: 733
id_lesion: 1183
id_filename: 3416
midas_path: 17
midas_path: {'malignant-bcc': 608, 'benign-melanocytic nevus': 578, 'nan': 497, 'benign-other': 421, 'benign-seborrheic keratosis': 242, 'malignant-melanoma': 238, 'malignant-scc': 203, 'malignant-ak': 191, 'malignant-sccis': 165, 'other-melanocytic lesion, possible re-excision (severe, spitz, aimp)': 109, 'benign-dermatofibroma': 51, 'other-non-neoplastic, inflammatory, infectious': 39, 'benign-hemangioma': 30, 'benign-fibrous papule': 18, 'malignant-other': 14, 'melanocytic tumor, possible re-excision (severe, spitz, aimp)': 6, 'unknown': 6}
y16_description: 16
y16: 16
y16: {'Basal Cell Carcinoma': 608, 'Melanocytic Nevus': 578, 'Other Benign': 421, 'Seborrheic Keratosis': 242, 'Melanoma': 238, 'Squamous Cell Carcinoma': 203, 'Actinic Keratosis': 191, 'Squamous Cell Carcinoma In Situ': 165, 'Melanocytic Lesion': 109, 'Dermatofibroma': 51, 'Non-neoplastic': 39, 'Hemangioma': 30, 'Fibrous Papule': 18, 'Other Malignant': 1

In [4]:
# train_dataset[0]['x']
train_dataset[0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3024x4032>,
 'y': {'y3': 'malignant',
  'y16': 'Squamous Cell Carcinoma In Situ',
  'y16_description': 'Squamous Cell Carcinoma In Situ - Early stage SCC, confined to epidermis.',
  'x_skintype': 'white skin',
  'x_skincolor': 'white',
  'x_skintone': 'medium',
  'x_location': 'torso',
  'text_full': 'The patient has white skin. The patient has white skin color. The patient has medium skin tone. The lesion is on the torso. The lesion is malignant. The patient is diagnosed with squamous cell carcinoma in situ On pathology, focal squamous cell carcinoma, in situ, \xa0 arising in background actinic keratosis.',
  'text_outcome': 'The lesion is malignant. The patient is diagnosed with squamous cell carcinoma in situ On pathology, focal squamous cell carcinoma, in situ, \xa0 arising in background actinic keratosis.',
  'text_y3': 'The lesion is malignant.',
  'text_y16': 'The patient is diagnosed with squamous cell carcinoma i

In [5]:
# # read train.parquet
# tmp_df = pd.read_parquet("train.parquet")
# tmp_df.head()
# tmp_df['choices'].value_counts()
# tmp_df['idx_choices'].value_counts()


df = pd.concat([train_df, val_df, test_df], ignore_index=True)
df.reset_index(drop=True, inplace=True)
df['uid'] = df.index + 1
df['uid'] = df['uid'].astype(int)
df['patient_id'] = df['id_patient']
df['lesion_id'] = df['id_lesion']
df['lesion_images'] = df['id_lesion_images']
df['image_path'] = "data/images/midas/" + df['id_actual_filename']
df['question'] = "Is the lesion malignant, benign, or other?"
df['answer'] = df['y3']
df['rationale'] = df['text_x_skincolor'] + " " +  df['text_x_skintone'] + " " + df['text_x_location']
df['choices'] = "malignant; benign; other; unknown"
df['idx_choices'] = "(A) malignant\n(B) benign\n(C) other\n(D) unknown"
df.head()

keep_cols = ['uid', 'patient_id', 'lesion_id', 'lesion_images', 'image_path', 'question', 'answer', 'rationale', 'choices', 'idx_choices']
other_cols = df.columns[~df.columns.isin(keep_cols)]
cols2remove = ["text_full", "text_outcome", "id_actual_filename", "id_filename"]
df = df[keep_cols + list(other_cols[~other_cols.isin(cols2remove)])]

# df.to_parquet("data_share/midas_share.parquet", index=False)
df


,uid,patient_id,lesion_id,lesion_images,image_path,question,answer,rationale,choices,idx_choices,...,x_location,id_lesion_images,text_y3,text_y16,text_x_skintype,text_x_skincolor,text_x_skintone,text_x_location,text_demo,text_lesion
0,1,1,1_chest_no,"[1ft, 6in, dscope]",data/images/midas/s-prd-398966407.jpg,"Is the lesion malignant, benign, or other?",malignant,The patient has white skin color. The patient ...,malignant; benign; other; unknown,(A) malignant\n(B) benign\n(C) other\n(D) unknown,...,torso,"[1ft, 6in, dscope]",The lesion is malignant.,The patient is diagnosed with squamous cell ca...,The patient has white skin.,The patient has white skin color.,The patient has medium skin tone.,The lesion is on the torso.,The patient is 80-year-old white female with F...,"The lesion is on the chest, imaged at 1ft. It ..."
1,2,1,1_chest_no,"[1ft, 6in, dscope]",data/images/midas/s-prd-398966642.jpg,"Is the lesion malignant, benign, or other?",malignant,The patient has white skin color. The patient ...,malignant; benign; other; unknown,(A) malignant\n(B) benign\n(C) other\n(D) unknown,...,torso,"[1ft, 6in, dscope]",The lesion is malignant.,The patient is diagnosed with squamous cell ca...,The patient has white skin.,The patient has white skin color.,The patient has medium skin tone.,The lesion is on the torso.,The patient is 80-year-old white female with F...,"The lesion is on the chest, imaged at 6in. It ..."
2,3,1,1_chest_no,"[1ft, 6in, dscope]",data/images/midas/s-prd-398966845.jpg,"Is the lesion malignant, benign, or other?",malignant,The patient has white skin color. The patient ...,malignant; benign; other; unknown,(A) malignant\n(B) benign\n(C) other\n(D) unknown,...,torso,"[1ft, 6in, dscope]",The lesion is malignant.,The patient is diagnosed with squamous cell ca...,The patient has white skin.,The patient has white skin color.,The patient has medium skin tone.,The lesion is on the torso.,The patient is 80-year-old white female with F...,"The lesion is on the chest, imaged at dscope. ..."
3,4,1,1_l lower back_no,"[1ft, dscope]",data/images/midas/s-prd-398967381.jpg,"Is the lesion malignant, benign, or other?",benign,The patient has white skin color. The patient ...,malignant; benign; other; unknown,(A) malignant\n(B) benign\n(C) other\n(D) unknown,...,torso,"[1ft, dscope]",The lesion is benign.,The patient is diagnosed with melanocytic nevus,The patient has white skin.,The patient has white skin color.,The patient has medium skin tone.,The lesion is on the torso.,The patient is 80-year-old white female with F...,"The lesion is on the left lower back, imaged a..."
4,5,1,1_l lower back_no,"[1ft, dscope]",data/images/midas/s-prd-398967587.jpg,"Is the lesion malignant, benign, or other?",benign,The patient has white skin color. The patient ...,malignant; benign; other; unknown,(A) malignant\n(B) benign\n(C) other\n(D) unknown,...,torso,"[1ft, dscope]",The lesion is benign.,The patient is diagnosed with melanocytic nevus,The patient has white skin.,The patient has white skin color.,The patient has medium skin tone.,The lesion is on the torso.,The patient is 80-year-old white female with F...,"The lesion is on the left lower back, imaged a..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3352,3353,807,807_right calf_no,"[6in, dscope, 1ft, dscope]",data/images/midas/s-prd-820554156.jpg,"Is the lesion malignant, benign, or other?",other,The patient has white skin color. The patient ...,malignant; benign; other; unknown,(A) malignant\n(B) benign\n(C) other\n(D) unknown,...,leg,"[6in, dscope, 1ft, dscope]",The lesion is other.,The patient is diagnosed with melanocytic lesion,The patient has white skin.,The patient has white skin color.,The patient has medium skin tone.,The lesion is on the leg.,The patient is 65-year-old white male with Fit...,"The lesion is on the right calf, imaged at 1ft..."
3353,3354,807,807_right calf_no,"[6in, dscope, 1ft, dscope]",data/images/midas/s-prd-820558468.jpg,"I

In [7]:
print("=== Parquet Summary ===")
print(f"Unique images (rows):   {df['uid'].nunique()}")
print(f"Unique lesions:         {df['lesion_id'].nunique()}")
print(f"Unique patients:        {df['patient_id'].nunique()}")

print(f"\n=== Lesion image availability ===")
lesion_combo = df.groupby('lesion_id')['lesion_distance'].apply(
    lambda x: tuple(sorted(set(x)))
).value_counts()
for combo, count in lesion_combo.items():
    print(f"  {str(list(combo)):40s}  {count} lesions")

print(f"\n=== Class distribution (y3) ===")
lesion_y3 = df.groupby('lesion_id')['y3'].first().value_counts()
for cls, count in lesion_y3.items():
    print(f"  {cls:12s}  {count} lesions")


=== Parquet Summary ===
Unique images (rows):   3357
Unique lesions:         1181
Unique patients:        733

=== Lesion image availability ===
  ['1ft', '6in', 'dscope']                  974 lesions
  ['virtual']                               135 lesions
  ['6in', 'dscope']                         24 lesions
  ['1ft', '6in', 'dscope', 'virtual']       19 lesions
  ['1ft', 'dscope']                         14 lesions
  ['dscope']                                8 lesions
  ['1ft', '6in']                            4 lesions
  ['1ft']                                   2 lesions
  ['6in', 'virtual']                        1 lesions

=== Class distribution (y3) ===
  malignant     466 lesions
  benign        442 lesions
  other         273 lesions


In [ ]:
import json

data_dict = {
    "uid": "Unique ID per image.",
    "patient_id": "Patient identifier from midas_record_id.",
    "lesion_id": "Unique lesion identifier: concat of midas_record_id, midas_location, and midas_iscontrol.",
    "lesion_images": "List of available image distances for this lesion (e.g. ['6in', 'dscope', '1ft']).",
    "image_path": "Path to the lesion image file.",
    "question": "Fixed question: 'Is the lesion malignant, benign, or other?'",
    "answer": "Three-level outcome: malignant, benign, or other.",
    "choices": "Answer options: 'malignant; benign; other; unknown'.",
    "idx_choices": "Indexed choices: '(A) malignant\\n(B) benign\\n(C) other\\n(D) unknown'.",
    "rationale": "Natural language description combining skin color (white/brown), skin tone (light/medium/dark), and lesion location (head/neck/torso/arm/hand/leg/foot/pelvic/other).",
    "y3": "Three-level diagnosis: malignant, benign, other.",
    "y16": "16-level diagnosis: Basal Cell Carcinoma, Melanocytic Nevus, Other Benign, Seborrheic Keratosis, Melanoma, Squamous Cell Carcinoma, Actinic Keratosis, Squamous Cell Carcinoma In Situ, Melanocytic Lesion, Dermatofibroma, Non-neoplastic, Hemangioma, Fibrous Papule, Other Malignant, Melanocytic Tumor, Unknown.",
    "y16_description": "Full description of the 16-level diagnosis with clinical details.",
    "x_skintype": "7 levels: light white skin, white skin, dark white skin, light brown skin, brown skin, dark brown skin, unknown.",
    "x_skincolor": "Skin color: white, brown, unknown.",
    "x_skintone": "Skin tone: light, medium, dark, unknown.",
    "x_location": "Body region: head, neck, torso, arm, hand, leg, foot, pelvic, other.",
    "demo_gender": "Patient gender: male, female, unknown.",
    "demo_age": "Patient age.",
    "demo_fitzpatrick_skintype": "Fitzpatrick skin type (I-VI scale).",
    "demo_melanoma_history": "Prior melanoma history: yes, no, unknown.",
    "demo_ethnicity": "Hispanic/Latino ethnicity: yes, no, unknown.",
    "demo_race": "Race: white, asian, other, unknown, black or african american, american indian or alaska native.",
    "lesion_distance": "Image capture distance: 6in, dscope, 1ft, virtual.",
    "lesion_location": "Detailed lesion location on body (407 unique values).",
    "lesion_length_mm": "Lesion length in mm (0.0-80.0).",
    "lesion_width_mm": "Lesion width in mm (0.0-80.0).",
    "text_*": "Natural language text columns generated from tabular features using template.",
}

# with open("data_share/midas_share_dictionary.json", "w") as f:
#     json.dump(data_dict, f, indent=2)
